In [1]:
import time

In [2]:


# Read CSV text file
dev = [s.strip().split(", ") for s in open('hw1-data/income.dev.txt').readlines()]

train = [s.strip().split(", ") for s in open('hw1-data/income.train.txt').readlines()]

In [18]:

# IMPORT DATA FROM LAST WEEKS SOLUTION

import sys
from collections import defaultdict
import numpy as np


def process_data(filename):
    X, Y = [], []
    for j, line in enumerate(open(filename)):
        line = line.strip()
        features = line.split(", ")
        feat_vec = np.zeros(dimension)
        for i, fv in enumerate(features[:-1]): # last one is target
            if i in numerical_fields: # two numerical fields
                feat_vec[feature_map[i, 0]] = float(fv) / 50  # NB: diff 2 not 1!
            elif (i, fv) in feature_map: # ignore unobserved features
                feat_vec[feature_map[i, fv]] = 1

        X.append(feat_vec)

        Y.append(1 if features[-1].upper() == ">50K" else -1) # fake for testdata
    
    return np.array(X), np.array(Y)



field_value_freqs = defaultdict(lambda : defaultdict(int)) # field_id -> value -> freq
numerical_fields = [] # [] for binarizing all the fields
for line in open("hw1-data/income.train.txt"):
    line = line.strip()
    features = line.split(", ")[:-1] # exclude target label
    for i, fv in enumerate(features):
        field_value_freqs[i][0 if i in numerical_fields else fv] += 1

feature_map = {}
feature_remap = {}
for i, value_freqs in field_value_freqs.items():
    for v in value_freqs:
        k = len(feature_map) # bias
        feature_map[i, v] = k
        feature_remap[k] = i, v

dimension = len(feature_map) # bias
print("dimensionality: %d" % dimension) #, feature_map

train_data = process_data("hw1-data/income.train.txt")
dev_data = process_data("hw1-data/income.dev.txt")



dimensionality: 230


In [6]:
# cleaning

## HOW DO I IMPLEMENT AVERAGE?

# DEVELOP DATA TO PASS TO perc() FUNCTION
data_train_tupe = []
for i, row in enumerate(zip(train_data[0],train_data[1])):
    row_vect = np.append(row[0],[1.]) # ADD STUFF TO VECTOR HERE
    data_train_tupe.append((row_vect,row[1])) 
    
data_dev_tupe = []
for i, row in enumerate(zip(dev_data[0],dev_data[1])):
    row_vect = np.append(row[0],[1.]) # ADD STUFF TO VECTOR HERE
    data_dev_tupe.append((row_vect,row[1]))
    
# ADD BIAS BY ADDING 1 AT END OF BINARIZED VECTOR

# TRAIN MODEL
def perc(data, dev_data, method = 'vanilla'):
    weight = np.zeros(len(data[0][0]))
    weight_s = np.zeros(len(data[0][0]))
    avgw = np.array([0.,0.,0.])
    epochs = 5
    supp_vec = set()
    best_margin = 10000
    
    for e in range(epochs):
        k = 0
        error = 0
        match = 0
        positives = 0
        #tavgw = np.array([0.,0.,0.])
        
        for i, (x, y) in enumerate(data):
            
            prediction = np.dot(x,weight)
            
            # POSITIVES
            if (prediction >= 0): #and y >= 0) :
                positives += 1
                
            # POSITIVE MATCH
            if (prediction >= 0 and y >= 0) :
                match += 1
                
            # NEG MATCH
            elif (prediction < 0 and y < 0):
                match += 1
                
            # NO MATCH
            else:
                error += 1
            
            if prediction * y <= 0:
                weight += x * y
                k += 1
                
            # update ws (AVERAGE PERCEPTRON METHOD)
            weight_s += weight 
            
        if method == 'vanilla':
            dev_err, dev_pos = test_perc(weight, dev_data) # CHANGE PASS FROM WEIGHT TO WEIGHT_S FOR AVG VS NO AVERAGE

        else:
            dev_err, dev_pos = test_perc(weight_s, dev_data) # CHANGE PASS FROM WEIGHT TO WEIGHT_S FOR AVG VS NO AVERAGE

        
        print('epoch', e+1, 'updates', k, f'({round((positives/len(data)*100),2)}%)', 'dev_err', f'{round(dev_err,2)}%', f'(+:{round(dev_pos,2)}%)')
        
        if dev_err < best_margin:
            best_margin = dev_err
            best_weight = weight_s
        
    print('\nBest Dev Error:', f'{round(best_margin,2)}%')
    return best_weight
        
          
# TEST MODEL
def test_perc(weights_s, df_bin, phase = 'normal'):
    '''takes weights previously calculated and applies them to binarized dataframe. Then calculates positive percent and error rates'''        
    k = 0
    error = 0
    match = 0
    positives = 0
      
    for i, (x, y) in enumerate(df_bin):
          
        prediction = np.dot(x,weights_s)
            
        # POSITIVES
        if (prediction > 0): #and y >= 0) :
            positives += 1
                
        # POSITIVE MATCH
        if (prediction >= 0 and y >= 0) :
            match += 1
                
        # NEG MATCH
        elif (prediction < 0 and y < 0):
            match += 1
                
        # NO MATCH
        else:
            error += 1
        
        
    dev_err = error/len(df_bin)*100
    dev_pos = positives/len(df_bin)*100
    
    return dev_err, dev_pos
        
# CALL PERC FUNCTION        

time1 = time.time()

print('Vanilla')
best_weight_van = perc(data_train_tupe, data_dev_tupe)

print('\nAverage')
best_weight_avg = perc(data_train_tupe, data_dev_tupe, method = 'average')

time2 = time.time()
print('duration is:', time2-time1, 'seconds.')


Vanilla
epoch 1 updates 1257 (26.48%) dev_err 22.4% (+:27.5%)
epoch 2 updates 1221 (26.64%) dev_err 20.8% (+:25.4%)
epoch 3 updates 1177 (26.22%) dev_err 18.0% (+:21.5%)
epoch 4 updates 1170 (26.2%) dev_err 19.7% (+:12.3%)
epoch 5 updates 1172 (26.22%) dev_err 18.7% (+:17.7%)

Best Dev Error: 18.0%

Average
epoch 1 updates 1257 (26.48%) dev_err 15.0% (+:18.6%)
epoch 2 updates 1221 (26.64%) dev_err 15.1% (+:19.3%)
epoch 3 updates 1177 (26.22%) dev_err 14.8% (+:20.0%)
epoch 4 updates 1170 (26.2%) dev_err 14.7% (+:19.3%)
epoch 5 updates 1172 (26.22%) dev_err 14.8% (+:20.0%)

Best Dev Error: 14.7%
duration is: 0.27571988105773926 seconds.


In [10]:
# CALCUALTE TOP AND BOTTOM

top_bott = []
for i, w in enumerate(best_weight_avg):
    top_bott.append((i,w))
    
top_bott_sorted = sorted(top_bott, key = lambda x: x[1], reverse = True)
for i in range(5):
    # POSITIVE
    for key, vals in feature_map.items():
        if vals == top_bott_sorted[i][0]:
            feat_val = key
    print('\nPositive Index:', top_bott_sorted[i][0], 'Positive Weight:', top_bott_sorted[i][1], 'Positive Feature:', feat_val)
    
    neg_i = -i -1
    # NEGATIVE
    for key, vals in feature_map.items():
        if vals == top_bott_sorted[neg_i][0]:
            feat_val = key
    print('Neg Index:', top_bott_sorted[neg_i][0], 'Neg Weight:', top_bott_sorted[neg_i][1], 'Neg Feature:', feat_val)


Positive Index: 85 Positive Weight: 178980.0 Positive Feature: (2, 'Doctorate')
Neg Index: 3 Neg Weight: -150069.0 Neg Feature: (0, '28')

Positive Index: 90 Positive Weight: 151494.0 Positive Feature: (3, 'Married-civ-spouse')
Neg Index: 230 Neg Weight: -146333.0 Neg Feature: (3, 'Married-civ-spouse')

Positive Index: 83 Positive Weight: 137016.0 Positive Feature: (2, 'Prof-school')
Neg Index: 81 Neg Weight: -141763.0 Neg Feature: (2, '7th-8th')

Positive Index: 201 Positive Weight: 121924.0 Positive Feature: (8, 'Iran')
Neg Index: 104 Neg Weight: -140683.0 Neg Feature: (4, 'Farming-fishing')

Positive Index: 148 Positive Weight: 102122.0 Positive Feature: (7, '65')
Neg Index: 46 Neg Weight: -124170.0 Neg Feature: (0, '26')


# Redo using new, ordered data


In [11]:
# CREATE ORDERED CSV 
train_rev = sorted(train, key=lambda x: x[-1]) 

np.savetxt('train_ordered.txt', train_rev, delimiter = ', ', fmt = '% s')

train_data_new = process_data("train_ordered.txt")


In [12]:
data_train_tupe_new = []
for i, row in enumerate(zip(train_data_new[0],train_data_new[1])):
    row_vect = np.append(row[0],[1.]) # ADD STUFF TO VECTOR HERE
    data_train_tupe_new.append((row_vect,row[1])) 
    

data_dev_tupe_new = []
for i, row in enumerate(zip(dev_data[0],dev_data[1])):
    row_vect = np.append(row[0],[1.]) # ADD STUFF TO VECTOR HERE
    data_dev_tupe_new.append((row_vect,row[1]))
    
    
print('Vanilla')
weight = perc(data_train_tupe_new, data_dev_tupe_new)

print('\nAverage')
weight = perc(data_train_tupe_new, data_dev_tupe_new, method = 'Average')

Vanilla
epoch 1 updates 4 (25.02%) dev_err 76.4% (+:100.0%)
epoch 2 updates 8 (25.02%) dev_err 76.4% (+:100.0%)
epoch 3 updates 9 (25.04%) dev_err 76.3% (+:98.9%)
epoch 4 updates 9 (25.06%) dev_err 76.4% (+:100.0%)
epoch 5 updates 10 (25.08%) dev_err 76.3% (+:99.7%)

Best Dev Error: 76.3%

Average
epoch 1 updates 4 (25.02%) dev_err 27.8% (+:4.4%)
epoch 2 updates 8 (25.02%) dev_err 23.7% (+:0.3%)
epoch 3 updates 9 (25.04%) dev_err 23.6% (+:0.0%)
epoch 4 updates 9 (25.06%) dev_err 23.6% (+:0.0%)
epoch 5 updates 10 (25.08%) dev_err 23.6% (+:0.0%)

Best Dev Error: 23.6%


# Feature Engineering



In [13]:
# ADD ORIGINAL AGE AND HOURS
data_train_tupe_feature1 = []
for i, row in enumerate(zip(train_data[0],train_data[1])):
    row_vect = np.append(row[0], [1., float(train[i][0]), float(train[i][-3])]) # ADD STUFF TO VECTOR HERE
    data_train_tupe_feature1.append((row_vect,row[1])) 
    
data_dev_tupe_feature1 = []
for i, row in enumerate(zip(dev_data[0],dev_data[1])):
    row_vect = np.append(row[0], [1., float(dev[i][0]), float(dev[i][-3])]) # ADD STUFF TO VECTOR HERE
    data_dev_tupe_feature1.append((row_vect,row[1]))


print('Vanilla')
weight = perc(data_train_tupe_feature1, data_dev_tupe_feature1)

print('\nAverage')
weight = perc(data_train_tupe_feature1, data_dev_tupe_feature1, method = 'Average')

Vanilla
epoch 1 updates 1858 (27.42%) dev_err 23.8% (+:0.2%)
epoch 2 updates 1676 (26.54%) dev_err 23.7% (+:0.1%)
epoch 3 updates 1601 (26.24%) dev_err 18.6% (+:23.8%)
epoch 4 updates 1516 (26.02%) dev_err 19.6% (+:26.8%)
epoch 5 updates 1510 (25.9%) dev_err 23.4% (+:0.2%)

Best Dev Error: 18.6%

Average
epoch 1 updates 1858 (27.42%) dev_err 23.6% (+:0.0%)
epoch 2 updates 1676 (26.54%) dev_err 23.6% (+:0.0%)
epoch 3 updates 1601 (26.24%) dev_err 23.6% (+:0.0%)
epoch 4 updates 1516 (26.02%) dev_err 23.5% (+:0.1%)
epoch 5 updates 1510 (25.9%) dev_err 22.7% (+:1.5%)

Best Dev Error: 22.7%


In [14]:
# CONVERT TO ZERO MEAN

train_mean = np.mean(train_data[0], axis = 0)


data_train_tupe_mean = []
for i, row in enumerate(zip(train_data[0] ,train_data[1])):
    row_vect = np.append(row[0]- train_mean,[1.]) # ADD STUFF TO VECTOR HERE
    data_train_tupe_mean.append((row_vect,row[1])) 

data_dev_tupe_mean = []
for i, row in enumerate(zip(dev_data[0],dev_data[1])):
    row_vect = np.append(row[0] - train_mean,[1.]) # ADD STUFF TO VECTOR HERE
    data_dev_tupe_mean.append((row_vect,row[1]))
        
    
print('Vanilla')
weight_s = perc(data_train_tupe_mean, data_dev_tupe_mean)

print('\nAverage')
best_weight_mean = perc(data_train_tupe_mean, data_dev_tupe_mean, method = 'average')



Vanilla
epoch 1 updates 1177 (25.08%) dev_err 22.9% (+:30.1%)
epoch 2 updates 1121 (25.04%) dev_err 20.9% (+:25.1%)
epoch 3 updates 1150 (25.02%) dev_err 22.0% (+:26.0%)
epoch 4 updates 1107 (25.0%) dev_err 25.3% (+:33.7%)
epoch 5 updates 1129 (25.04%) dev_err 23.6% (+:25.4%)

Best Dev Error: 20.9%

Average
epoch 1 updates 1177 (25.08%) dev_err 14.1% (+:20.1%)
epoch 2 updates 1121 (25.04%) dev_err 14.5% (+:20.5%)
epoch 3 updates 1150 (25.02%) dev_err 14.5% (+:21.1%)
epoch 4 updates 1107 (25.0%) dev_err 14.8% (+:21.0%)
epoch 5 updates 1129 (25.04%) dev_err 15.4% (+:21.4%)

Best Dev Error: 14.1%


In [15]:
# STANDARD DEVIATION
train_std = np.std(train_data[0], axis=0)


data_train_std = []
for i, row in enumerate(zip(train_data[0],train_data[1])):
    row_vect = np.append(row[0]/train_std,[1.]) # ADD STUFF TO VECTOR HERE
    data_train_std.append((row_vect,row[1])) 

    
data_dev_tupe_std = []
for i, row in enumerate(zip(dev_data[0],dev_data[1])):
    row_vect = np.append(row[0]/train_std,[1.]) # ADD STUFF TO VECTOR HERE
    data_dev_tupe_std.append((row_vect,row[1]))
    
    
    
print('Vanilla')
weight_s = perc(data_train_std, data_dev_tupe_std)

print('\nAverage')
weight_s = perc(data_train_std, data_dev_tupe_std, method = 'average')

Vanilla
epoch 1 updates 1162 (25.24%) dev_err 22.8% (+:26.6%)
epoch 2 updates 1091 (25.0%) dev_err 23.7% (+:28.9%)
epoch 3 updates 1134 (25.06%) dev_err 23.3% (+:29.1%)
epoch 4 updates 1128 (24.98%) dev_err 25.3% (+:35.9%)
epoch 5 updates 1119 (25.08%) dev_err 20.2% (+:19.2%)

Best Dev Error: 20.2%

Average
epoch 1 updates 1162 (25.24%) dev_err 16.5% (+:21.3%)
epoch 2 updates 1091 (25.0%) dev_err 16.2% (+:20.6%)
epoch 3 updates 1134 (25.06%) dev_err 15.8% (+:21.2%)
epoch 4 updates 1128 (24.98%) dev_err 16.1% (+:21.1%)
epoch 5 updates 1119 (25.08%) dev_err 15.7% (+:21.5%)

Best Dev Error: 15.7%


In [16]:
# COMBINATION
train_std = np.std(train_data[0], axis=0)


data_train_test = []
for i, row in enumerate(zip(train_data[0],train_data[1])):
    row_vect = np.append(row[0]/train_std,[1., float(train[i][0]), float(train[i][-3])]) # ADD STUFF TO VECTOR HERE
    data_train_test.append((row_vect,row[1])) 

    
data_dev_tupe_test = []
for i, row in enumerate(zip(dev_data[0],dev_data[1])):
    row_vect = np.append(row[0]/train_std,[1., float(dev[i][0]), float(dev[i][-3])]) # ADD STUFF TO VECTOR HERE
    data_dev_tupe_test.append((row_vect,row[1]))
    
    
    
print('Vanilla')
weight_s = perc(data_train_test, data_dev_tupe_test)

print('\nAverage')
weight_s = perc(data_train_test, data_dev_tupe_test, method = 'average')

Vanilla
epoch 1 updates 1546 (26.14%) dev_err 23.1% (+:0.7%)
epoch 2 updates 1295 (25.32%) dev_err 17.6% (+:28.0%)
epoch 3 updates 1210 (25.18%) dev_err 16.0% (+:22.8%)
epoch 4 updates 1186 (25.22%) dev_err 19.0% (+:6.6%)
epoch 5 updates 1158 (25.14%) dev_err 19.6% (+:34.4%)

Best Dev Error: 16.0%

Average
epoch 1 updates 1546 (26.14%) dev_err 18.1% (+:9.3%)
epoch 2 updates 1295 (25.32%) dev_err 16.6% (+:13.6%)
epoch 3 updates 1210 (25.18%) dev_err 16.1% (+:15.5%)
epoch 4 updates 1186 (25.22%) dev_err 16.2% (+:17.0%)
epoch 5 updates 1158 (25.14%) dev_err 15.8% (+:18.2%)

Best Dev Error: 15.8%


# Predict on Blind Data

In [17]:
blind = [s.strip().split(", ") for s in open('hw1-data/income_blind.txt').readlines()]

new_data = []

best_weight_mean = best_weight_mean # DECLARED UP ABOVE USING AVERAGED METHOD EPOCH 1 WEIGHT


#BINARIZE Blind
for row in blind:
    new_row = []
    for j, x in enumerate(row):
        feature = (j, x)
        try:
            new_row.append(feature_map[feature])
        except:
            print('new feature:', feature, 'cannot be analysed. I"m OK with that.')
    new_data.append(new_row)
    
bindata_blind = np.zeros((1000, len(feature_map)))
bindata_blind

for i, row in enumerate(new_data):
    for x in row:
        bindata_blind[i][x] = 1


data_blind_tupe = []
for row in bindata_blind:
    row_vect = np.append(row,[1.]) # ADD STUFF TO VECTOR HERE
    data_blind_tupe.append(row_vect)


neg = 0
match = 0
positives = 0
labels = []
      
for i, x in enumerate(data_blind_tupe):
         
    prediction = np.dot(x,best_weight_avg)  # COULDN'T MAKE IT WORK USING THE TRANSFORMED AVERAGED DATA, DECIDED TO JUST USE AVERAGE WEIGHT VECTOR INSTEAD. STILL 14.7% ACCURACY.
            
    # POSITIVES
    if (prediction > 0):
        positives += 1
        labels.append('>50K')
                
                
    # NO MATCH
    else:
        neg += 1
        labels.append('<=50K')
        
        
neg = neg/len(data_blind_tupe)*100
pos = positives/len(data_blind_tupe)*100


print(neg, pos)



with open('income.test.predicted.txt', 'w') as file:
    for i,row in enumerate(blind):
        line = row
        line.append(labels[i])
        file.write(', '.join(line) + '\n')

new feature: (7, '31') cannot be analysed. I"m OK with that.
new feature: (0, '83') cannot be analysed. I"m OK with that.
new feature: (0, '82') cannot be analysed. I"m OK with that.
new feature: (7, '29') cannot be analysed. I"m OK with that.
new feature: (0, '82') cannot be analysed. I"m OK with that.
new feature: (8, 'Hungary') cannot be analysed. I"m OK with that.
79.60000000000001 20.4
